In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [2]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, :])
    return data[:, :-1], data[:, -1]  # X, y

In [3]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
X[0], y[0]

(array([5.1, 3.5, 1.4, 0.2]), 0.0)

In [5]:
X[1], y[1]

(array([4.9, 3. , 1.4, 0.2]), 0.0)

In [6]:
X[10], y[10]

(array([5.4, 3.7, 1.5, 0.2]), 0.0)

In [7]:
# 特征的可能性被假设为高斯分布
# 高斯模型假设这些一个特征的所有属于某个类别的观测值符合高斯分布
class NaiveBayes(object):
    def __init__(self):
        self.model = None
        
    # 数学期望
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))
    
    # 标准差
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X)))
    
    # 概率密度函数
    def gaussian_prob(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
    
    # 处理X_train
    def summarize(self, train_data):
        return [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
    
    # 分类别求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y))
        data = {label: [] for label in labels}
        for f, label in zip(X, y):
            data[label].append(f)
        self.model = {label: self.summarize(value) for label, value in data.items()}
        print ('GaussianNB train done!')
        
    # 计算概率
    def calculate_prob(self, input_data):
        probs = {}
        for label, value in self.model.items():
            probs[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probs[label] *= self.gaussian_prob(input_data[i], mean, stdev)
        return probs
    
    # 预测类别
    def predict(self, X_test):
        return sorted(self.calculate_prob(X_test).items(), key=lambda x: x[-1])[-1][0]
        
    def score(self, X_test, y_test):
        right_count = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right_count += 1
                
        return right_count / float(len(X_test))

In [8]:
model = NaiveBayes()

In [9]:
model.fit(X_train, y_train)

GaussianNB train done!


In [10]:
print(model.predict([4.9, 3. , 1.4, 0.2]))

0.0


In [11]:
model.score(X_test, y_test)

1.0

In [12]:
from sklearn.naive_bayes import GaussianNB

In [13]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [14]:
clf.score(X_test, y_test)

1.0

In [15]:
clf.predict([[4.4,  3.2,  1.3,  0.2]])

array([0.])